<img src="images/taxi.png" width="500">

Let's assume Smartcab is the only vehicle in this parking lot. We can break up the parking lot into a 5x5 grid, which gives us 25 possible taxi locations. These 25 locations are one part of our state space. Notice the current location state of our taxi is coordinate (3, 1).

You'll also notice there are four (4) locations that we can pick up and drop off a passenger: R, G, Y, B or [(0,0), (0,4), (4,0), (4,3)] in (row, col) coordinates. Our illustrated passenger is in location Y and they wish to go to location R.

When we also account for one (1) additional passenger state of being inside the taxi, we can take all combinations of passenger locations and destination locations to come to a total number of states for our taxi environment; there's four (4) destinations and five (4 + 1) passenger locations.


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Generating-environment" data-toc-modified-id="Generating-environment-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Generating environment</a></span><ul class="toc-item"><li><span><a href="#States" data-toc-modified-id="States-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>States</a></span></li><li><span><a href="#Actions" data-toc-modified-id="Actions-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Actions</a></span></li></ul></li><li><span><a href="#Generating-Random-Policy" data-toc-modified-id="Generating-Random-Policy-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Generating Random Policy</a></span></li><li><span><a href="#Training-algorithm" data-toc-modified-id="Training-algorithm-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Training algorithm</a></span><ul class="toc-item"><li><span><a href="#Q-Learning" data-toc-modified-id="Q-Learning-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Q Learning</a></span></li><li><span><a href="#Q-table-result" data-toc-modified-id="Q-table-result-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Q table result</a></span></li></ul></li><li><span><a href="#Evaluate-Algorithm" data-toc-modified-id="Evaluate-Algorithm-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Evaluate Algorithm</a></span></li></ul></div>

# Imports

In [ ]:
import gym
import numpy as np
import random

# Generating environment

In [25]:
env = gym.make("Taxi-v3").env

env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



## States


So, our taxi environment has 5×5×5×4=500 total possible states.
    * 5x5 of the grid word.
    * 5 Passenger locations (4 Points + inside cab).
    * 4 possible destinations.

## Actions

The agent encounters one of the 500 states and it takes an action. The action in our case can be to move in a direction or decide to pickup/dropoff a passenger.

In other words, we have six possible actions:

    0 = south
    1 = north
    2 = east
    3 = west
    4 = pickup
    5 = dropoff

In [27]:
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space Discrete(6)
State Space Discrete(500)


In [28]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)

In [29]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

# Generating Random Policy

env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    # sample() method will return as a random action from the set of all possible actions.
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

print_frames(frames)

# Training algorithm
## Q Learning

We'll initialize the Q-table to a 500×6 matrix of zeros:

In [30]:
# Q table creation
q_table = np.zeros([env.observation_space.n, env.action_space.n])
q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [31]:
%%time
"""Training the agent"""


# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 10000):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

clear_output(wait=True)
print(f"Episode: {i+1}")
print("Training finished.\n")

Episode: 10000
Training finished.

Wall time: 8.46 s


## Q table result

In [33]:
# We will show the q value for state 328
q_table[328]

array([-2.34423776, -2.27325277, -2.34632068, -2.33718465, -7.63895629,
       -6.27775996])

We can observe that in state 328, the best action will be the one of the position 1, which refers to 

# Evaluate Algorithm

In [40]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 10

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False

    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 10 episodes:
Average timesteps per episode: 12.5
Average penalties per episode: 0.0


In [45]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = np.argmax(q_table[state])
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 12
Penalties incurred: 0


In [46]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 12
State: 0
Action: 5
Reward: 20
